In [1]:
import requests
from bs4 import BeautifulSoup as bs
import os
import json
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [3]:
from time import sleep
import re
from selenium import webdriver
import getpass

In [4]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State

In [6]:
TEMPLATE = 'plotly_white'

app = dash.Dash(__name__)
app.layout = html.Div(
    children = [
        html.Div([        
            html.Br(),
            dbc.Row([
                dbc.Col(html.H1(children="DMS points analysis"), width=5),
                dbc.Col(width=5),
                    ], justify='center'),
        dcc.Input(id="ID", type="text", placeholder=""),
        dcc.Input(id="PW", type="password", placeholder="")]),
        html.Button('Submit', id='submit-val', n_clicks=0),
        dbc.Row(
            [
                dbc.Row([
                dbc.Col(
                    html.Div([
                        dcc.Graph(id='graph1'),
                        dcc.Graph(id='graph3')
                ]), width=4)]),
                dbc.Row([
                    dbc.Col(
                            html.Div(id='feedback'), width=9
                    ),
                    dbc.Col(width=2)
                ], justify='center'),
                html.Br(),
                dbc.Col(
                    html.Div([
                        dcc.Graph(id='graph2')
                ]), width=4), 
            ])
]
)


@app.callback(
    [Output(component_id='graph1', component_property='figure'),
     Output(component_id='graph2', component_property='figure'),
     Output(component_id='feedback', component_property='children'),
     Output(component_id='graph3', component_property='figure'),
    ],
    Input('submit-val', 'n_clicks'), 
    State('ID', 'value'), 
    State('PW', 'value')
)


def update_output(n_clicks, value_id, value_pw):
    ID = value_id
    PW = value_pw
    driver = webdriver.Chrome('/Users/user/Downloads/\chromedriver_win32/chromedriver')

    driver.implicitly_wait(3)

    driver.get('https://www.dsm-dms.com/')
    driver.find_element_by_xpath('//*[@id="meal"]/div/header/nav/button').click()
    driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/div[2]/div[1]/input[1]').send_keys(ID)
    driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/div[2]/div[1]/input[2]').send_keys(PW)
    driver.find_element_by_xpath('//*[@id="root"]/div[1]/div/div[2]/div[1]/button').click()
    sleep(1)

    at = driver.switch_to.alert
    at.accept()

    driver.get('https://www.dsm-dms.com/mypage')
    sleep(1)

    driver.find_element_by_xpath('//*[@id="root"]/div/div/div/div/div[2]/div[1]').click()
    html_c = driver.page_source
    soup = bs(html_c, 'html.parser')
    content = soup.select('.point--card--left--reason')
    date = soup.select('.point--card--left--date')
    points = soup.select('.point--card--point')
    
    df = pd.DataFrame(columns = ['content', 'date', 'points', 'sign'])
    for i in range(len(content)):
        df.loc[i] = [re.sub('<.+?>', '', str(content[i]), 0).strip(), re.sub('<.+?>', '', str(date[i]), 0).strip(), 
                 re.sub('<.+?>', '', str(points[i]), 0).strip(), re.sub('<.+?>', '', str(points[i]), 0).strip()[-2::-1]]
    
    for idx, point in enumerate(df['points']):
        if '--' in point:
            df.at[idx, 'points'] = point.strip('--')
            print(df.iloc[idx]['points'])
        if '+-' in point:
                print(idx)
                df.drop(idx, inplace = True)
    df['points'] = df['points'].astype('int64')
    df['date'] = pd.to_datetime(df['date'])
    
    demerit = abs(df[df['sign']=='-']['points'].groupby(df['content']).sum().sort_values()).reset_index()
    print(demerit)
    penalty = abs(df[df['sign']=='-']['points'].groupby(df['date']).sum()).reset_index()
    print(penalty)
    merit = abs(df[df['sign']=='+']['points'].groupby(df['content']).sum().sort_values()).reset_index()
    print(demerit)
    advantage = abs(df[df['sign']=='+']['points'].groupby(df['date']).sum()).reset_index()
    print(penalty)
    figure1 = {
        'data':[
            {'x': demerit['content'], 'y': demerit['points'], 'type': 'bar', 'name': 'Series1'}
        ],
        'layout':{
            'title': '벌점 비중', 'width' : 500, 'height' : 500
        }
    }
    
    figure2 = {
        'data':[
            {'x':penalty['date'], 'y': penalty['points'], 'type':'bar', 'name':'Series2'}
        ],
        'layout':{
            'title': '날짜별 벌점'
        }
    }
    
    figure3 = {
        'data':[
            {'x':merit['content'], 'y':merit['points'], 'type':'bar', 'name':'Series3'}
        ],
        'layout':{
            'title': '상점 비중', 'width' : 500, 'height' : 500
        }
    }
    

    return [figure1, figure2, '{}을(를) 조심하세요'.format(demerit[:1]['content'].values[0]), figure3]

 
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Jun/2021 09:45:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2021 09:45:09] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [04/Jun/2021 09:45:09] "GET /_dash-dependencies HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\users\user\anaconda3\envs\bert\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\users\user\anaconda3\envs\bert\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\users\user\anaconda3\envs\bert\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "c:\users\user\anaconda3\envs\bert\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "c:\users\user\anaconda3\envs\bert\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\users\user\anaconda3\envs\bert\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "c:\users\user\anaconda3\envs\bert\lib\site-pa

127.0.0.1 - - [04/Jun/2021 09:45:13] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [04/Jun/2021 09:45:24] "POST /_dash-update-component HTTP/1.1" 200 -


3
5
         content  points
0  주말 식사시간 무단 외출       5
1      기숙사 무단 출입       3
2        마스크 미착용       2
3        아침점호 지각       2
4   외출외박 후 귀사 지각       1
5        타 호실 출입       1
        date  points
0 2021-03-07       1
1 2021-03-10       6
2 2021-03-17       1
3 2021-03-21       1
4 2021-03-28       1
5 2021-04-03       1
6 2021-05-20       3
         content  points
0  주말 식사시간 무단 외출       5
1      기숙사 무단 출입       3
2        마스크 미착용       2
3        아침점호 지각       2
4   외출외박 후 귀사 지각       1
5        타 호실 출입       1
        date  points
0 2021-03-07       1
1 2021-03-10       6
2 2021-03-17       1
3 2021-03-21       1
4 2021-03-28       1
5 2021-04-03       1
6 2021-05-20       3
